In [1]:
import os
import torch
import librosa
import numpy as np
from gtts import gTTS
import torch, json, os
import soundfile as sf
from tokenizers import Tokenizer
from dataclasses import dataclass
from collections import defaultdict
from datasets import load_dataset, Audio
from typing import Any, Dict, List, Union
from datasets import Audio, Dataset, Value, Features, load_dataset
from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech, SpeechT5HifiGan

In [2]:
processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_tts")
model = SpeechT5ForTextToSpeech.from_pretrained("microsoft/speecht5_tts")
vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan")

c:\Users\thiro\anaconda3\envs\myenv\lib\site-packages\huggingface_hub\file_download.py:137: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\thiro\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [3]:
def SinhalaTTSdataset(mapping_json = 'data/tts/file-mapping.json'):
    with open(mapping_json) as f:
        mapping = json.load(f)
    
    data = {}
    data["audio"] = []
    data["normalized_text"] = []
    for val_dict in mapping.values():
        audio_path = f"data/tts/wavs/{val_dict['newfn']}"
        sinhala_text = val_dict['sinhala']
        if os.path.exists(audio_path):
            audio, sampling_rate = librosa.load(audio_path, sr=16000)
            audio_data = {
                            "path": audio_path,
                            "array": audio,
                            "sampling_rate": sampling_rate
                         }
            data["audio"].append(audio_data)
            data["normalized_text"].append(sinhala_text)

    return Dataset.from_dict(data)

In [4]:
try:
    dataset = SinhalaTTSdataset()
except:
    dataset = load_dataset(
                        "facebook/voxpopuli", 
                        "nl", split="train"
                        )

In [6]:
dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))
tokenizer = processor.tokenizer

MemoryError: 

In [7]:
def extract_all_chars(batch):
    all_text = " ".join(batch["normalized_text"])
    vocab = list(set(all_text))
    return {"vocab": [vocab], "all_text": [all_text]}

vocabs = dataset.map(
    extract_all_chars, 
    batched=True, 
    batch_size=-1, 
    keep_in_memory=True, 
    remove_columns=dataset.column_names,
)

replacements = [
                ('à', 'a'),
                ('ç', 'c'),
                ('è', 'e'),
                ('ë', 'e'),
                ('í', 'i'),
                ('ï', 'i'),
                ('ö', 'o'),
                ('ü', 'u'),
                ]

def cleanup_text(inputs):
    for src, dst in replacements:
        inputs["normalized_text"] = inputs["normalized_text"].replace(src, dst)
    return inputs

dataset = dataset.map(cleanup_text)

dataset_vocab = set(vocabs["vocab"][0])
tokenizer_vocab = {k for k,_ in tokenizer.get_vocab().items()}

Map:   0%|          | 0/3300 [00:00<?, ? examples/s]

Map:   0%|          | 0/3300 [00:00<?, ? examples/s]

NameError: name 'tokenizer' is not defined

In [ ]:
def create_speaker_embedding(waveform):
    with torch.no_grad():
                try:
                    speaker_embeddings = speaker_model.encode_batch(torch.tensor(waveform))
                    speaker_embeddings = torch.nn.functional.normalize(speaker_embeddings, dim=2)
                    speaker_embeddings = speaker_embeddings.squeeze().cpu().numpy()
                except:
                    speaker_embeddings = np.random.rand(512, )
    return speaker_embeddings

In [ ]:
def prepare_dataset(example):
    # load the audio data; if necessary, this resamples the audio to 16kHz
    audio = example["audio"]

    # feature extraction and tokenization
    example = processor(
        text=example["normalized_text"],
        audio_target=audio["array"], 
        sampling_rate=audio["sampling_rate"],
        return_attention_mask=False,
    )

    # strip off the batch dimension
    example["labels"] = example["labels"][0]

    # use SpeechBrain to obtain x-vector
    example["speaker_embeddings"] = create_speaker_embedding(audio["array"])

    return example

In [ ]:
dataset = dataset.map(
    prepare_dataset, remove_columns=dataset.column_names,
)

def is_not_too_long(input_ids):
    input_length = len(input_ids)
    return input_length < 200

dataset = dataset.filter(is_not_too_long, input_columns=["input_ids"])

Map:   0%|          | 0/3300 [00:00<?, ? examples/s]

Filter:   0%|          | 0/3300 [00:00<?, ? examples/s]

In [ ]:
dataset = dataset.train_test_split(test_size=0.1)

In [ ]:
@dataclass
class TTSDataCollatorWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:

        input_ids = [{"input_ids": feature["input_ids"]} for feature in features]
        label_features = [{"input_values": feature["labels"]} for feature in features]
        speaker_features = [feature["speaker_embeddings"] for feature in features]

        # collate the inputs and targets into a batch
        batch = processor.pad(
            input_ids=input_ids,
            labels=label_features,
            return_tensors="pt",
        )        

        # replace padding with -100 to ignore loss correctly
        batch["labels"] = batch["labels"].masked_fill(
            batch.decoder_attention_mask.unsqueeze(-1).ne(1), -100
        )

        # not used during fine-tuning
        del batch["decoder_attention_mask"]

        # round down target lengths to multiple of reduction factor
        if model.config.reduction_factor > 1:
            target_lengths = torch.tensor([
                len(feature["input_values"]) for feature in label_features
            ])
            target_lengths = target_lengths.new([
                length - length % model.config.reduction_factor for length in target_lengths
            ])
            max_length = max(target_lengths)
            batch["labels"] = batch["labels"][:, :max_length]

        # also add in the speaker embeddings
        batch["speaker_embeddings"] = torch.tensor(speaker_features)

        return batch

In [ ]:
data_collator = TTSDataCollatorWithPadding(processor=processor)

features = [
    dataset["train"][0],
    dataset["train"][1],
    dataset["train"][20],
]

batch = data_collator(features)

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
model.config.use_cache = False

In [ ]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
                                        output_dir="models/sinhala-text-to-speech",
                                        per_device_train_batch_size=2,
                                        gradient_accumulation_steps=2,
                                        learning_rate=1e-5,
                                        warmup_steps=500,
                                        max_steps=150,
                                        gradient_checkpointing=True,
                                        fp16=False,
                                        evaluation_strategy="steps",
                                        per_device_eval_batch_size=2,
                                        save_steps=10,
                                        eval_steps=10,
                                        logging_steps=25,
                                        report_to=["tensorboard"],
                                        load_best_model_at_end=True,
                                        greater_is_better=False,
                                        label_names=["labels"],
                                        push_to_hub=False,
                                    )

In [ ]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
                        args=training_args,
                        model=model,
                        train_dataset=dataset["train"],
                        eval_dataset=dataset["test"],
                        data_collator=data_collator,
                        tokenizer=processor.tokenizer,
                        )
trainer.train()

  0%|          | 0/150 [00:00<?, ?it/s]

  0%|          | 0/165 [00:00<?, ?it/s]

{'eval_loss': 0.8853644132614136, 'eval_runtime': 314.1971, 'eval_samples_per_second': 1.05, 'eval_steps_per_second': 0.525, 'epoch': 0.01}


  0%|          | 0/165 [00:00<?, ?it/s]

{'eval_loss': 0.8724579811096191, 'eval_runtime': 327.1562, 'eval_samples_per_second': 1.009, 'eval_steps_per_second': 0.504, 'epoch': 0.03}
{'loss': 0.9561, 'learning_rate': 5.000000000000001e-07, 'epoch': 0.03}


...

  0%|          | 0/165 [00:00<?, ?it/s]

{'eval_loss': 0.6363164782524109, 'eval_runtime': 303.684, 'eval_samples_per_second': 1.087, 'eval_steps_per_second': 0.543, 'epoch': 0.19}
{'loss': 0.7281, 'learning_rate': 3e-06, 'epoch': 0.2}


  0%|          | 0/165 [00:00<?, ?it/s]

{'eval_loss': 0.6351116895675659, 'eval_runtime': 289.4955, 'eval_samples_per_second': 1.14, 'eval_steps_per_second': 0.57, 'epoch': 0.2}
{'train_runtime': 7062.0068, 'train_samples_per_second': 0.085, 'train_steps_per_second': 0.021, 'train_loss': 0.8333395258585612, 'epoch': 0.2}


TrainOutput(global_step=150, training_loss=0.8333395258585612, metrics={'train_runtime': 7062.0068, 'train_samples_per_second': 0.085, 'train_steps_per_second': 0.021, 'train_loss': 0.8333395258585612, 'epoch': 0.2})